# Save Serializer Data to the Database

## Basic Usage

In [1]:
from datetime import date

from django.test import RequestFactory

from organizer.models import Tag
from organizer.serializers import StartupSerializer, TagSerializer

In [2]:
fake_request = RequestFactory().get("/api/v1/tag/")

In [3]:
data = {"name": "new tag"}
s_tag = TagSerializer(
    data=data,
    # request is necessary for full URL reversal
    # needed by url field of HyperlinkedModelSerializer
    context={"request": fake_request},
)

In [4]:
s_tag.is_valid()

True

In [5]:
s_tag.errors

{}

In [6]:
Tag.objects.filter(name="new tag").exists()

False

In [7]:
s_tag.save()

<Tag: new tag>

In [8]:
Tag.objects.filter(name="new tag").exists()

True

## Errors and Extra Values

In [9]:
s_tag = TagSerializer(
    data={},
    context={"request": fake_request},
)

In [10]:
s_tag.is_valid()

False

In [11]:
s_tag.errors

{'name': [ErrorDetail(string='This field is required.', code='required')]}

In [12]:
s_tag = TagSerializer(
    data={
        "name": 'newer tag',
        "ignored": "this value is ignored"
    },
    context={"request": fake_request},
)

In [13]:
s_tag.is_valid()

True

In [14]:
s_tag.errors

{}

In [15]:
s_tag.save()

<Tag: newer tag>

## Nested Serializer Handling

In [16]:
s_startup = StartupSerializer(
    data=dict(
        name="JamBon Software LLC",
        slug="jambon-software-llc",
        description="Consulting & training for web and mobile products.",
        founded_date=date(2013, 1, 13),
        contact="django@jambonsw.com",  # not a real email
        website="https://jambonsw.com",
        tags=[
            {"name": "newest tag"},
        ],
    ),
    context={"request": RequestFactory().get("/api/v1/startup/")},
)

In [17]:
s_startup.is_valid()

True

In [18]:
s_startup.errors

{}

In [19]:
# serializers inside serializers is not supported
# we will therefore need to modify all our serializers
# that use other serializers for model relations:
# StartupSerializer, NewsLinkSerializer, PostSerializer
try:
    s_startup.save()
except AssertionError as err:
    print(err)

The `.create()` method does not support writable nested fields by default.
Write an explicit `.create()` method for serializer `organizer.serializers.StartupSerializer`, or set `read_only=True` on nested serializer fields.


## Notebook Cleanup

In [20]:
Tag.objects.filter(name__endswith=" tag").delete()

(2, {'blog.Post_tags': 0, 'organizer.Startup_tags': 0, 'organizer.Tag': 2})